In [ ]:
from torchsummary import summary
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import torch.utils.data as Data
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import scipy.io

# Models

In [ ]:
class EEGNet(nn.Module):
    # __init__ part is to define what the layer/ model would contain 
    #      (set attributes, add layers or parameters of layer/model)
    # forward is to control the (training) dataflow or operation on data during training
    
    def __init__(self):
        super(EEGNet, self).__init__()
        
        self.F1 = 8  # level one filter count 
        self.F2 = 16 # level two filter count
        self.D = 2   # depthwise multiplier for doing depthwise convolution

        self.conv1 = nn.Sequential( 
            nn.Conv2d(1, self.F1, (1, 64), padding=(0, 32), bias=False),
            nn.BatchNorm2d(self.F1)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(self.F1, self.D*self.F1, (22, 1), groups=self.F1, bias=False),
            nn.BatchNorm2d(self.D*self.F1),
            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(0.5)
        )

        self.Conv3 = nn.Sequential(
            nn.Conv2d(self.D*self.F1, self.D*self.F1, (1, 16), padding=(0, 8), groups=self.D*self.F1, bias=False),
            nn.Conv2d(self.D*self.F1, self.F2, (1, 1), bias=False),
            nn.BatchNorm2d(self.F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(0.5)
        )

        self.classifier = nn.Linear(16*17, 4, bias=True) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.Conv3(x)
        
        x = x.view(-1, 16*17)
        x = self.classifier(x)
        return x

In [ ]:
class ShallowConvNet(nn.Module):
    def __init__(self):
        super(ShallowConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 40, (1, 13), bias=False)
        self.conv2 = nn.Conv2d(40, 40, (22, 1), bias=False)
        self.Bn1   = nn.BatchNorm2d(40)
        self.AvgPool1 = nn.AvgPool2d((1, 35), stride=(1, 7))
        self.Drop1 = nn.Dropout(0.25)
        self.classifier = nn.Linear(40*74, 4, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.Bn1(x)
        x = x ** 2
        x = self.AvgPool1(x)
        x = torch.log(x)
        x = self.Drop1(x)
        x = x.view(-1, 40*74)
        x = self.classifier(x)
        return x

In [ ]:
class SCCNet(nn.Module):
    def __init__(self):
        super(SCCNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 22, (22, 1))
        self.Bn1 = nn.BatchNorm2d(22)
        self.conv2 = nn.Conv2d(22, 20, (1, 12), padding=(0, 6))
        self.Bn2   = nn.BatchNorm2d(20)
        self.Drop1 = nn.Dropout(0.5)
        self.AvgPool1 = nn.AvgPool2d((1, 62), stride=(1, 12))
        self.classifier = nn.Linear(840, 4, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.Bn1(x)
        x = self.conv2(x)
        x = self.Bn2(x)
        x = x ** 2
        x = self.Drop1(x)
        x = self.AvgPool1(x)
        x = torch.log(x)
        x = x.view(-1, 840)
        x = self.classifier(x)

        return x

In [ ]:
class TSception(nn.Module):
    def __init__(self):
        super(TSception, self).__init__()
        self.pool = 8
        self.Tception1 = nn.Sequential(
            nn.Conv2d(1, 9, (1,int(0.5 * 250))),
            nn.LeakyReLU(),
            nn.AvgPool2d((1, 8), (1, 8))
            )
        
        self.Tception2 = nn.Sequential(
            nn.Conv2d(1, 9, (1,int(0.25 * 250))),
            nn.LeakyReLU(),
            nn.AvgPool2d((1, 8), (1, 8))
            )
        self.Tception3 = nn.Sequential(
            nn.Conv2d(1, 9, (1,int(0.0125 * 250))),
            nn.LeakyReLU(),
            nn.AvgPool2d((1, 8), (1, 8))
            )

        self.Sception1 = nn.Sequential(
            nn.Conv2d(9, 6, (22,1)),
            nn.LeakyReLU(),
            nn.AvgPool2d((1, 2), (1, 2))
            )
        
        self.Sception2 = nn.Sequential(
            nn.Conv2d(9, 6, (11,1), (11,1)),
            nn.LeakyReLU(),
            nn.AvgPool2d((1, 2), (1, 2))
            )
        
        self.BN_t = nn.BatchNorm2d(9)
        self.BN_s = nn.BatchNorm2d(6)

        
        self.fc = nn.Sequential(
            nn.Linear(1674, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 4),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x1 = self.Tception1(x)
        x2 = self.Tception2(x)
        x3 = self.Tception3(x)
        x = torch.cat((x1,x2,x3), dim=-1)
        x = self.BN_t(x)

        x1 = self.Sception1(x)
        x2 = self.Sception2(x)
        x = torch.cat((x1, x2), dim=2)
        x = self.BN_s(x)
        x = x.view(-1, 1674)
        x = self.fc(x)
        return x

In [ ]:
class TSception_MI(nn.Module):
    def __init__(self):
        super(TSception_MI, self).__init__()
        self.Tception1 = nn.Sequential(
            nn.Conv2d(1, 8, (1,32)),
            nn.LeakyReLU(),
            nn.AvgPool2d((1, 4), (1, 4)),
            nn.Dropout(0.3),
            )
        self.Tception2 = nn.Sequential(
            nn.Conv2d(1, 8, (1,16)),
            nn.LeakyReLU(),
            nn.AvgPool2d((1, 4), (1, 4)),
            nn.Dropout(0.3),
            )

        self.Sception1 = nn.Sequential(
            nn.Conv2d(8, 8, (22,1)),
            nn.LeakyReLU(),
            nn.AvgPool2d((1, 8), (1, 8)),
            nn.Dropout(0.3),
            )
        
        self.BN_t = nn.BatchNorm2d(8)
        self.BN_s = nn.BatchNorm2d(8)
 
        self.fc = nn.Sequential(
            nn.Linear(264, 125),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Linear(125, 4),    
        )

    def forward(self, x):
        x1 = self.Tception1(x)
        x2 = self.Tception2(x)
        x = torch.cat((x1,x2), dim=-1)
        x = self.BN_t(x)
        
        x = self.Sception1(x)
        x = self.BN_s(x)
        x = x.view(-1, 264)
        x = self.fc(x)
        return x

# Dataloaders

## individual subject

In [ ]:
def getDataLoader_ID(sub = 1, batch = 32, shuffle = True):
    """
    individual scheme
    ------------------------------
    parameters: 
        sub: the subject num
        batch: batch size
        shuffle: shuffle data or not
    return:
        train dataloader (288),1,22,562
        test dataloader (288),1,22,562
    """
    
    print('\nconstructing ID dataloader')
    data_train = scipy.io.loadmat('BCICIV_2a_mat/BCIC_S{:0>2d}_T.mat'.format(sub))
    data_test = scipy.io.loadmat('BCICIV_2a_mat/BCIC_S{:0>2d}_E.mat'.format(sub))

    dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    x_train = torch.Tensor(data_train['x_train']).unsqueeze(1)
    y_train = torch.Tensor(data_train['y_train']).view(-1).long()
    x_test = torch.Tensor(data_test['x_test']).unsqueeze(1)
    y_test = torch.Tensor(data_test['y_test']).view(-1).long()

    print(f"x_train size is: {x_train.size()}")
    print(f"y_train size is: {y_train.size()}")
    print(f"x_test size is: {x_test.size()}")
    print(f"y_test size is: {y_test.size()}")

    if torch.cuda.is_available():
        dev = torch.device("cuda")
        num_worker = 0
        x_train = x_train.to(dev)
        y_train = y_train.to(dev)
        x_test = x_test.to(dev)
        y_test = y_test.to(dev)
    else:
        dev = torch.device("cpu")
        num_worker = 8

    train_dataset = Data.TensorDataset(x_train, y_train)
    test_dataset = Data.TensorDataset(x_test, y_test)

    trainloader = Data.DataLoader(
        dataset = train_dataset,
        batch_size = batch,
        shuffle = shuffle,
        num_workers = num_worker
    )
    testloader =  Data.DataLoader(
        dataset = test_dataset,
        batch_size = 1,
        shuffle = False,
        num_workers = num_worker,
    )
    return trainloader, testloader

## subject-independent

In [ ]:
def getDataLoader_SI(subject=1, batch = 32, shuffle = True):
     """
    subject independent scheme
    ------------------------------
    parameters: 
        sub: the testing subject num
        batch: batch size
        shuffle: shuffle data or not
    return:
        train dataloader (288*8),1,22,562
        test dataloader (288),1,22,562
    """
        
    print('\nconstructing SI dataloader')

    Tname='BCICIV_2a_mat/BCIC_S{:0>2d}_T.mat'.format(subject)
    Ename='BCICIV_2a_mat/BCIC_S{:0>2d}_E.mat'.format(subject)
    data_train = scipy.io.loadmat(Tname)
    data_test = scipy.io.loadmat(Ename)
    x_tra = torch.Tensor(data_train['x_train']).unsqueeze(1)
    y_tra = torch.Tensor(data_train['y_train']).view(-1).long()
    x_te = torch.Tensor(data_test['x_test']).unsqueeze(1)
    y_te = torch.Tensor(data_test['y_test']).view(-1).long()

    for sub in range(1,10):
        if sub==subject:
            continue
        else:
            Tname='BCICIV_2a_mat/BCIC_S{:0>2d}_T.mat'.format(sub)
            Ename='BCICIV_2a_mat/BCIC_S{:0>2d}_E.mat'.format(sub)
        
            T = scipy.io.loadmat(Tname)
            E = scipy.io.loadmat(Ename)
            x_tra = torch.cat((x_tra, torch.Tensor(T['x_train']).unsqueeze(1)), dim=0)
            y_tra = torch.cat((y_tra, torch.Tensor(T['y_train']).view(-1).long()), dim=0)
            x_te = torch.cat((x_te, torch.Tensor(E['x_test']).unsqueeze(1)), dim=0)
            y_te = torch.cat((y_te, torch.Tensor(E['y_test']).view(-1).long()), dim=0)

    
    # first 288 is subject 1, needs to isolate from training data or extract as testing data
    xT, yT = x_tra[288:], y_tra[288:] 
    xE, yE = x_te[288:], y_te[288:]
    x_train = torch.cat((xT, xE),dim=0)
    y_train = torch.cat((yT, yE), dim=0)
    x_test = x_te[:288]
    y_test = y_te[:288]

    print(f"x_train size is: {x_train.size()}") 
    print(f"y_train size is: {y_train.size()}")
    print(f"x_test size is: {x_test.size()}") 
    print(f"y_test size is: {y_test.size()}")

    if torch.cuda.is_available():
        dev = torch.device("cuda")
        num_worker = 0
        x_train = x_train.to(dev)
        y_train = y_train.to(dev)
        x_test = x_test.to(dev)
        y_test = y_test.to(dev)
    else:
        dev = torch.device("cpu")
        num_worker = 8
    
    
    train_dataset = Data.TensorDataset(x_train, y_train)
    test_dataset = Data.TensorDataset(x_test, y_test)

    trainloader = Data.DataLoader(
        dataset = train_dataset,
        batch_size = batch,
        shuffle = shuffle,
        num_workers = num_worker
    )
    testloader =  Data.DataLoader(
        dataset = test_dataset,
        batch_size = 1,
        shuffle = False,
        num_workers = num_worker,
    )

    return trainloader, testloader

## subject-dependent

In [ ]:
def getDataLoader_SD(subject=1, batch = 32, shuffle = True):
    """
    subject dependent scheme
    ------------------------------
    parameters: 
        sub: the testing subject num
        batch: batch size
        shuffle: shuffle data or not
    return:
        train dataloader (288*9),1,22,562
        test dataloader (288),1,22,562
    """
    
    print('\nconstructing SD dataloader')
    
    Tname='BCICIV_2a_mat/BCIC_S{:0>2d}_T.mat'.format(subject)
    Ename='BCICIV_2a_mat/BCIC_S{:0>2d}_E.mat'.format(subject)
    data_train = scipy.io.loadmat(Tname)
    data_test = scipy.io.loadmat(Ename)
    x_tra = torch.Tensor(data_train['x_train']).unsqueeze(1)
    y_tra = torch.Tensor(data_train['y_train']).view(-1).long()
    x_te = torch.Tensor(data_test['x_test']).unsqueeze(1)
    y_te = torch.Tensor(data_test['y_test']).view(-1).long()

    for sub in range(1,10):
        if sub==subject:
            continue
        else:
            Tname='BCICIV_2a_mat/BCIC_S{:0>2d}_T.mat'.format(sub)
            Ename='BCICIV_2a_mat/BCIC_S{:0>2d}_E.mat'.format(sub)
        
            T = scipy.io.loadmat(Tname)
            E = scipy.io.loadmat(Ename)
            x_tra = torch.cat((x_tra, torch.Tensor(T['x_train']).unsqueeze(1)), dim=0)
            y_tra = torch.cat((y_tra, torch.Tensor(T['y_train']).view(-1).long()), dim=0)
            x_te = torch.cat((x_te, torch.Tensor(E['x_test']).unsqueeze(1)), dim=0)
            y_te = torch.cat((y_te, torch.Tensor(E['y_test']).view(-1).long()), dim=0)

    # first 288 is subject 1, needs to extract as testing
    xE, yE = x_te[288:], y_te[288:]
    x_train = torch.cat((x_tra, xE),dim=0)
    y_train = torch.cat((y_tra, yE), dim=0)
    x_test = x_te[:288]
    y_test = y_te[:288]
 
    print(f"x_train size is: {x_train.size()}") 
    print(f"y_train size is: {y_train.size()}")
    print(f"x_test size is: {x_test.size()}") 
    print(f"y_test size is: {y_test.size()}")

    if torch.cuda.is_available():
        dev = torch.device("cuda")
        num_worker = 0
        x_train = x_train.to(dev)
        y_train = y_train.to(dev)
        x_test = x_test.to(dev)
        y_test = y_test.to(dev)
    else:
        dev = torch.device("cpu")
        num_worker = 8
    
    
    train_dataset = Data.TensorDataset(x_train, y_train)
    test_dataset = Data.TensorDataset(x_test, y_test)

    trainloader = Data.DataLoader(
        dataset = train_dataset,
        batch_size = batch,
        shuffle = shuffle,
        num_workers = num_worker
    )
    testloader =  Data.DataLoader(
        dataset = test_dataset,
        batch_size = 1,
        shuffle = False,
        num_workers = num_worker,
    )

    return trainloader, testloader

## subject-independent plus fine-tuning

In [ ]:
def getDataLoader_FT(subject=1, batch = 32, shuffle = True):
    """
    subject independent + fine-tuning scheme
    ------------------------------
    parameters: 
        subject: the testing subject num
        batch: batch size
        shuffle: shuffle data or not
    return:
        phase 1 train dataloader (288*8),1,22,562
        phase 2 train dataloader (288),1,22,562
        test dataloader (288),1,22,562
    """
    
    print('\nconstructing SI+FT dataloader')

    Tname='BCICIV_2a_mat/BCIC_S{:0>2d}_T.mat'.format(subject)
    Ename='BCICIV_2a_mat/BCIC_S{:0>2d}_E.mat'.format(subject)
    data_train = scipy.io.loadmat(Tname)
    data_test = scipy.io.loadmat(Ename)
    x_tra = torch.Tensor(data_train['x_train']).unsqueeze(1)
    y_tra = torch.Tensor(data_train['y_train']).view(-1).long()
    x_te = torch.Tensor(data_test['x_test']).unsqueeze(1)
    y_te = torch.Tensor(data_test['y_test']).view(-1).long()

    for sub in range(1,10):
        if sub==subject:
            continue
        else:
            Tname='BCICIV_2a_mat/BCIC_S{:0>2d}_T.mat'.format(sub)
            Ename='BCICIV_2a_mat/BCIC_S{:0>2d}_E.mat'.format(sub)
        
            T = scipy.io.loadmat(Tname)
            E = scipy.io.loadmat(Ename)
            x_tra = torch.cat((x_tra, torch.Tensor(T['x_train']).unsqueeze(1)), dim=0)
            y_tra = torch.cat((y_tra, torch.Tensor(T['y_train']).view(-1).long()), dim=0)
            x_te = torch.cat((x_te, torch.Tensor(E['x_test']).unsqueeze(1)), dim=0)
            y_te = torch.cat((y_te, torch.Tensor(E['y_test']).view(-1).long()), dim=0)

    
    # subject 2 - 9
    xT, yT = x_tra[288:], y_tra[288:] # T data
    xE, yE = x_te[288:], y_te[288:] # E data
    x_train = torch.cat((xT, xE),dim=0)
    y_train = torch.cat((yT, yE), dim=0)

    # subject 1, for fine-tuning training and testing
    x_train2 = x_tra[:288]
    y_train2 = y_tra[:288]
    x_test = x_te[:288]
    y_test = y_te[:288]

    print(f"x_train size is: {x_train.size()}") 
    print(f"y_train size is: {y_train.size()}")
    print(f"x_train2 size is: {x_train2.size()}") 
    print(f"y_train2 size is: {y_train2.size()}")
    print(f"x_test size is: {x_test.size()}") 
    print(f"y_test size is: {y_test.size()}")

    if torch.cuda.is_available():
        dev = torch.device("cuda")
        num_worker = 0
        x_train = x_train.to(dev)
        y_train = y_train.to(dev)
        x_train2 = x_train2.to(dev)
        y_train2 = y_train2.to(dev)
        x_test = x_test.to(dev)
        y_test = y_test.to(dev)
    else:
        dev = torch.device("cpu")
        num_worker = 8
    
    
    train_dataset = Data.TensorDataset(x_train, y_train)
    train2_dataset = Data.TensorDataset(x_train2, y_train2)
    test_dataset = Data.TensorDataset(x_test, y_test)

    trainloader = Data.DataLoader(
        dataset = train_dataset,
        batch_size = batch,
        shuffle = shuffle,
        num_workers = num_worker
    )
    trainloader2 = Data.DataLoader(
        dataset = train2_dataset,
        batch_size = batch,
        shuffle = shuffle,
        num_workers = num_worker
    )
    testloader =  Data.DataLoader(
        dataset = test_dataset,
        batch_size = 1,
        shuffle = False,
        num_workers = num_worker,
    )

    return trainloader, trainloader2, testloader

# Utility functions

In [ ]:
def train(net, dataloader, epo = 200, lr = 0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    print('\nstart training')
    net.train()
    for epoch in range(epo):
        for xb, yb in dataloader:
            pred = net(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        if (epoch + 1) % 10 == 0:
            print(f"epoch {epoch+1} loss: {loss.item()}")
            
def test(net, dataloader):
    acc = 0
    yTrue = []
    yPred = []
    
    print('\nstart testing')

    net.eval()
    for x, y in dataloader:
        pred = net(x)
        if pred.argmax() == y:
            acc += 1
            
        yTrue.append(y)
        yPred.append(pred.argmax())
    
    plotConfusionMatrix(yTrue, yPred ) 
    
    acc /= len(dataloader)

    print(f"accuracy: {acc}")

In [ ]:
# confusion matrix
def plotConfusionMatrix(y_true, y_pred, normalize=True):
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype("float")/cm.sum(axis=1)[:,np.newaxis]
    else:
        cm = cm.astype("float")/cm.sum()
    plt.figure(figsize=(20, 6))
    plt.imshow(cm, cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()

    plt.xticks(np.arange(4))
    plt.yticks(np.arange(5))
    plt.ylim(len(cm) - 0.5, -0.5)
    
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(x=j, y=i, s=("%.2f"%cm[i][j]), va='center', ha='center')
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Part1 problem1

In [ ]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") # determine current hardware

trainloader, testloader = getDataLoader_SI(batch = 32) # if not passing variables, default batch size 32
eegnet = EEGNet().to(dev)                              # assign the network to specified hardware

train(eegnet, trainloader, epo = 200, lr = 0.001)
test(eegnet, testloader) 

summary(eegnet,(1, 22, 562)) # View network details

# Part1 problem2

In [ ]:
# batch = 64
# lr = 0.003
# by adjusting the batch size and the learning rate, we can finish this problem.

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
trainloader, testloader = getDataLoader_SI(batch = 64)

eegnet = EEGNet().to(dev) 

train(eegnet, trainloader,epo = 200, lr = 0.003)
test(eegnet, testloader)

summary(eegnet,(1, 22, 562))

# Part1 problem3

Train with different schemes by using different dataloader functions

* subject-independent

In [ ]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
trainloader, testloader = getDataLoader_SI(batch = 32)

sccnet = SCCNet().to(dev) 

train(sccnet, trainloader,epo = 200, lr = 0.001)
test(sccnet, testloader)
summary(sccnet,(1, 22, 562))

* subject dependent

In [ ]:
trainloader, testloader = getDataLoader_SD(batch = 32)

sccnet = SCCNet().to(dev)

train(sccnet, trainloader,epo = 200, lr = 0.001)
test(sccnet, testloader)

* subject-independent plus fine-tuning

In [ ]:
trainloader, trainloader2, testloader = getDataLoader_FT(batch = 32)

sccnet = SCCNet().to(dev)

train(sccnet, trainloader,epo = 200, lr = 0.001)
train(sccnet, trainloader2, epo = 200, lr = 0.001)
test(sccnet, testloader)

# Part1 problem4

* For this part, we can just modify the parameter shuffle to False, and compare the results to previous results.

* for EEG individual training

In [ ]:
trainloader, testloader = getDataLoader_ID(batch = 32, shuffle = False)

eegnet = EEGNet().to(dev) 

train(eegnet, trainloader, epo = 200, lr = 0.001)
test(eegnet, testloader)

* for EEG subject dependent

In [ ]:
trainloader, testloader = getDataLoader_SD(batch = 32, shuffle = False)

eegnet = EEGNet().to(dev) 

train(eegnet, trainloader,epo = 200, lr = 0.001)
test(eegnet, testloader)

accuracy: 25.34

# Part2 problem1

* EEGnet

In [ ]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#trainloader, testloader = getDataLoader_SI(batch = 32, shuffle = True) 
#=================> if we run subject independent scheme
#trainloader, testloader = getDataLoader_ID(batch = 32, shuffle = True)
#=================> if we run within subject scheme
#trainloader, trainloader2, testloader = getDataLoader_FT(batch = 32, shuffle = True) 
#=================> if we run subject independent + fine-tuning scheme

trainloader, testloader = getDataLoader_subject_dependent(batch = 32, shuffle = True)

eegnet = EEGNet().to(dev)

train(eegnet, trainloader,epo = 200, lr = 0.001)
#train(eegnet, trainloader2,epo = 200, lr = 0.001) 
#=================> add this line if we run subject independent + fine-tuning scheme

test(eegnet, testloader)
summary(eegnet,(1, 22, 562))

* shallow convolution net

In [ ]:
#trainloader, testloader = getDataLoader_SI(batch = 32, shuffle = True) 
#=================> if we run subject independent scheme
#trainloader, testloader = getDataLoader_ID(batch = 32, shuffle = True)
#=================> if we run within subject scheme
#trainloader, trainloader2, testloader = getDataLoader_FT(batch = 32, shuffle = True) 
#=================> if we run subject independent + fine-tuning scheme

trainloader, testloader = getDataLoader_subject_dependent(batch = 32, shuffle = True)

ShallowConvNet = ShallowConvNet().to(dev)

train(ShallowConvNet, trainloader, epo = 200, lr = 0.001)
#train(ShallowConvNet, trainloader2,epo = 200, lr = 0.001) 
#=================> add this line if we run subject independent + fine-tuning scheme

test(ShallowConvNet, testloader)
summary(ShallowConvNet,(1, 22, 562))

* sccnet

In [ ]:
#trainloader, testloader = getDataLoader_SI(batch = 32, shuffle = True) 
#=================> if we run subject independent scheme
#trainloader, testloader = getDataLoader_ID(batch = 32, shuffle = True)
#=================> if we run within subject scheme
#trainloader, trainloader2, testloader = getDataLoader_FT(batch = 32, shuffle = True) 
#=================> if we run subject independent + fine-tuning scheme

trainloader, testloader = getDataLoader_subject_dependent(batch = 32, shuffle = True)

sccnet = SCCNet().to(dev)

train(sccnet, trainloader, epo = 200, lr = 0.001)
#train(sccnet, trainloader2,epo = 200, lr = 0.001) 
#=================> add this line if we run subject independent + fine-tuning scheme

test(sccnet, testloader)
summary(sccnet,(1, 22, 562))

* TSception

In [ ]:
#trainloader, testloader = getDataLoader_SI(batch = 32, shuffle = True) 
#=================> if we run subject independent scheme
#trainloader, testloader = getDataLoader_ID(batch = 32, shuffle = True)
#=================> if we run within subject scheme
#trainloader, trainloader2, testloader = getDataLoader_FT(batch = 32, shuffle = True) 
#=================> if we run subject independent + fine-tuning scheme

trainloader, testloader = getDataLoader_subject_dependent(batch = 32, shuffle = True)

tsception = TSception().to(dev)

train(tsception, trainloader, epo = 200, lr = 0.001)
#train(tsception, trainloader2,epo = 200, lr = 0.001) 
#=================> add this line if we run subject independent + fine-tuning scheme

test(tsception, testloader)
summary(tsception,(1, 22, 562))

* TSception we try to improve it

In [ ]:
#trainloader, testloader = getDataLoader_SI(batch = 32, shuffle = True) 
#=================> if we run subject independent scheme
#trainloader, testloader = getDataLoader_ID(batch = 32, shuffle = True)
#=================> if we run within subject scheme
#trainloader, trainloader2, testloader = getDataLoader_FT(batch = 32, shuffle = True) 
#=================> if we run subject independent + fine-tuning scheme

trainloader, testloader = getDataLoader_subject_dependent(batch = 32, shuffle = True)

tsception_mi = TSception_MI().to(dev)

train(tsception_mi, trainloader, epo = 200, lr = 0.001)
#train(tsception, trainloader2,epo = 200, lr = 0.001) 
#=================> add this line if we run subject independent + fine-tuning scheme

test(tsception, testloader)
summary(tsception_mi,(1, 22, 562))